# Python 内存管理

## 内存管理架构

![](https://img3.doubanio.com/view/ark_works_pic/common-largeshow/public/179342.jpg)

## 数据结构

### block 数据结构

```c
typedef uchar block;
```

### pool 数据结构

```c
/* Pool for small blocks. */
struct pool_header {
    union { block *_padding;
            uint count} ref;       /* number of allocated blocks */
    block *freeblock;              /* pool's free list head */
    struct pool_header *nextpool;  /* next pool of this size class */
    struct pool_header *prevpool;  /* previous pool of this size class */
    uint arenaindex;               /* index into arenas of base adr */
    uint szidx;                    /* block size class index */
    uint nextoffset;               /* bytes to virgin block */
    uint maxnextoffset;            /* largest valid nextoffset */
};
```

### arena 数据结构

```c
struct arena_object {
    /* The address of the arena, as returned by malloc.  Note that 0
     * will never be returned by a successful malloc, and is used
     * here to mark an arena_object that doesn't correspond to an
     * allocated arena.
     */
    uptr address;
    
    /* Pool-aligned pointer to the next pool to be carbed off. */
    block* pool_address;
    
    /* The number of available pools in the arena: free pools + never-allocated pools */
    uint nfreepools;
    
    /* The total number of pools in the arena, whether or not available. */
    uint ntotalpools;
    
    /* Singly-linked list of available pools. */
    struct pool_header* freepools;
    
    /* Whenever this arena_object is not assosicated with an allocated
     * arena, the nextarena member is used to link all unassociated 
     * arena_objects in the single-linked `unused_arena_objects` list.
     * The prevarena member is unused in this case.
     * 
     * When this arena_object is associated with an allocated arena
     * with at least one available pool, both members are used in the 
     * doubly-linked `usable_arenas` list, which is maintained in increasing
     * order of `nfreepools` values.
     *
     * Else this arena_object is associated with an allocated arena all of 
     * whose pools are in use. `nextarena` and `prevarena` are both meaningless
     * in this case.
     */
    struct arena_object *nextarena;
    struct arena_object *prevarena;
};
```

## pool

pool内存空间布局

![](https://img3.doubanio.com/view/ark_works_pic/common-largeshow/public/179373.jpg)

## arena

arena内存空间布局

![](https://img1.doubanio.com/view/ark_works_pic/common-largeshow/public/180219.jpg)

pool与arena内存空间布局对比

![](https://img1.doubanio.com/view/ark_works_pic/common-largeshow/public/180217.jpg)

arenas, usable_arenas, unused_arena_objects

![](https://img1.doubanio.com/view/ark_works_pic/common-largeshow/public/180218.jpg)

## PyObject_Malloc 函数概览

`Objects/obmalloc.c`

```c
void* PyObject_Malloc(size_t nbytes)
{
    block *bp;
    poolp pool;
    poolp next;
    uint size;
    
    // 如果申请的内存小于SMALL_REQUEST_THRESHOLD, 使用Python的小块内存的内存池，否则，转向malloc
    if ((nbytes - 1) < SMALL_REQUEST_THRESHOLD) {
        // 根据申请内存的大小获得对应的size class index
        size = (unit)(nbytes - 1) >> ALIGNMENT_SHIFT;
        pool = usedpools[size + size];
        // 如果usedpools中有可用的pool，使用这个pool来分配block
        if (pool != pool-> nextpool) {
            // 在pool中分配block，分配结束后，如果pool中的block都被分配了，将pool从usedpools中摘除
            next = pool->nextpool;
            pool = pool-prepool;
            next->prevpool = pool;
            pool->nextpool = next;
            return (void*)bp;
        }
    }
    
    //usedpools中没有可用的pool，从usable_arenas中获取pool
    if (usable_arenas == NULL) {
        //usable_arenas中没有可用的arena，开始申请arena
        usable_arenas = new_arena();
        usable_arenas->nextarena = usable_arenas->prevarena = NULL;
    }
    
    //从usable_arenas的第一个arena中获取一个pool
    pool = usable_arenas->freepools;
    if (pool != NULL) {

init_pool:
        //获取pool成功，惊醒init pool的动作，将pool放入used_pools中，并返回分配的block
        ......
            
        goto init_pool;
    }

redirect:
    //如果申请的内存不小于SMALL_REQUEST_THRESHOLD，使用malloc
    if (nbytes == 0):
        nbytes = 1
    return (void*)malloc(nbytes);
}
```

## 小块内存池全景

![](https://img9.doubanio.com/view/ark_works_pic/common-largeshow/public/180245.jpg)